In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valid_size = 256

# network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # dropout (probability to keep units)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
# store weights & biases
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [8]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [10]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [11]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size], keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1, batch + 1, loss, valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images[:test_valid_size], y: mnist.test.labels[:test_valid_size],
                keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch  1, Batch   1 -Loss: 58596.0625 Validation Accuracy: 0.140625
Epoch  1, Batch   2 -Loss: 51518.9453 Validation Accuracy: 0.125000
Epoch  1, Batch   3 -Loss: 42649.6484 Validation Accuracy: 0.109375
Epoch  1, Batch   4 -Loss: 36027.1094 Validation Accuracy: 0.128906
Epoch  1, Batch   5 -Loss: 27843.9727 Validation Accuracy: 0.144531
Epoch  1, Batch   6 -Loss: 27966.6445 Validation Accuracy: 0.152344
Epoch  1, Batch   7 -Loss: 25078.4199 Validation Accuracy: 0.148438
Epoch  1, Batch   8 -Loss: 24183.7520 Validation Accuracy: 0.171875
Epoch  1, Batch   9 -Loss: 21770.2891 Validation Accuracy: 0.175781
Epoch  1, Batch  10 -Loss: 20014.9180 Validation Accuracy: 0.187500
Epoch  1, Batch  11 -Loss: 19039.1133 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 17450.2656 Validation Accuracy: 0

Epoch  1, Batch 116 -Loss:  4316.4521 Validation Accuracy: 0.601562
Epoch  1, Batch 117 -Loss:  3746.9834 Validation Accuracy: 0.605469
Epoch  1, Batch 118 -Loss:  3788.8623 Validation Accuracy: 0.613281
Epoch  1, Batch 119 -Loss:  4317.0972 Validation Accuracy: 0.601562
Epoch  1, Batch 120 -Loss:  3594.8286 Validation Accuracy: 0.605469
Epoch  1, Batch 121 -Loss:  4699.1685 Validation Accuracy: 0.613281
Epoch  1, Batch 122 -Loss:  4344.1421 Validation Accuracy: 0.601562
Epoch  1, Batch 123 -Loss:  3969.2241 Validation Accuracy: 0.597656
Epoch  1, Batch 124 -Loss:  4987.0879 Validation Accuracy: 0.609375
Epoch  1, Batch 125 -Loss:  4271.0415 Validation Accuracy: 0.636719
Epoch  1, Batch 126 -Loss:  3007.7568 Validation Accuracy: 0.625000
Epoch  1, Batch 127 -Loss:  4261.0791 Validation Accuracy: 0.636719
Epoch  1, Batch 128 -Loss:  3724.8960 Validation Accuracy: 0.628906
Epoch  1, Batch 129 -Loss:  3734.6584 Validation Accuracy: 0.625000
Epoch  1, Batch 130 -Loss:  4672.1182 Validation

Epoch  1, Batch 237 -Loss:  2688.2097 Validation Accuracy: 0.691406
Epoch  1, Batch 238 -Loss:  1963.9058 Validation Accuracy: 0.695312
Epoch  1, Batch 239 -Loss:  2432.8738 Validation Accuracy: 0.691406
Epoch  1, Batch 240 -Loss:  1830.3824 Validation Accuracy: 0.699219
Epoch  1, Batch 241 -Loss:  2084.7266 Validation Accuracy: 0.695312
Epoch  1, Batch 242 -Loss:  1892.6050 Validation Accuracy: 0.687500
Epoch  1, Batch 243 -Loss:  2350.1250 Validation Accuracy: 0.691406
Epoch  1, Batch 244 -Loss:  2101.1011 Validation Accuracy: 0.699219
Epoch  1, Batch 245 -Loss:  2459.9114 Validation Accuracy: 0.699219
Epoch  1, Batch 246 -Loss:  2254.5637 Validation Accuracy: 0.703125
Epoch  1, Batch 247 -Loss:  2287.2778 Validation Accuracy: 0.707031
Epoch  1, Batch 248 -Loss:  1931.9600 Validation Accuracy: 0.703125
Epoch  1, Batch 249 -Loss:  2678.5352 Validation Accuracy: 0.703125
Epoch  1, Batch 250 -Loss:  1743.9211 Validation Accuracy: 0.703125
Epoch  1, Batch 251 -Loss:  2175.0374 Validation

Epoch  1, Batch 358 -Loss:  1565.9886 Validation Accuracy: 0.753906
Epoch  1, Batch 359 -Loss:  2105.2361 Validation Accuracy: 0.753906
Epoch  1, Batch 360 -Loss:  1889.9489 Validation Accuracy: 0.761719
Epoch  1, Batch 361 -Loss:  1710.1042 Validation Accuracy: 0.765625
Epoch  1, Batch 362 -Loss:  2516.2949 Validation Accuracy: 0.769531
Epoch  1, Batch 363 -Loss:  1585.8728 Validation Accuracy: 0.757812
Epoch  1, Batch 364 -Loss:  1606.5015 Validation Accuracy: 0.765625
Epoch  1, Batch 365 -Loss:  1795.1810 Validation Accuracy: 0.761719
Epoch  1, Batch 366 -Loss:  1569.5636 Validation Accuracy: 0.750000
Epoch  1, Batch 367 -Loss:  1735.1202 Validation Accuracy: 0.765625
Epoch  1, Batch 368 -Loss:  1536.1051 Validation Accuracy: 0.761719
Epoch  1, Batch 369 -Loss:  2001.1405 Validation Accuracy: 0.757812
Epoch  1, Batch 370 -Loss:  1938.8130 Validation Accuracy: 0.757812
Epoch  1, Batch 371 -Loss:  1838.1309 Validation Accuracy: 0.750000
Epoch  1, Batch 372 -Loss:  1225.2516 Validation

Epoch  2, Batch  50 -Loss:  1522.4941 Validation Accuracy: 0.765625
Epoch  2, Batch  51 -Loss:  1095.1700 Validation Accuracy: 0.773438
Epoch  2, Batch  52 -Loss:  1944.0945 Validation Accuracy: 0.773438
Epoch  2, Batch  53 -Loss:   861.1417 Validation Accuracy: 0.769531
Epoch  2, Batch  54 -Loss:  2036.6344 Validation Accuracy: 0.781250
Epoch  2, Batch  55 -Loss:  1407.8776 Validation Accuracy: 0.781250
Epoch  2, Batch  56 -Loss:  1540.3491 Validation Accuracy: 0.769531
Epoch  2, Batch  57 -Loss:   686.6462 Validation Accuracy: 0.781250
Epoch  2, Batch  58 -Loss:   780.5243 Validation Accuracy: 0.773438
Epoch  2, Batch  59 -Loss:  1526.1447 Validation Accuracy: 0.777344
Epoch  2, Batch  60 -Loss:  1054.1946 Validation Accuracy: 0.781250
Epoch  2, Batch  61 -Loss:  1162.3926 Validation Accuracy: 0.777344
Epoch  2, Batch  62 -Loss:  1184.0896 Validation Accuracy: 0.781250
Epoch  2, Batch  63 -Loss:  1339.1816 Validation Accuracy: 0.785156
Epoch  2, Batch  64 -Loss:  1122.4667 Validation

...

Epoch  9, Batch 314 -Loss:   320.7819 Validation Accuracy: 0.832031
Epoch  9, Batch 315 -Loss:   335.1613 Validation Accuracy: 0.832031
Epoch  9, Batch 316 -Loss:   341.7507 Validation Accuracy: 0.832031
Epoch  9, Batch 317 -Loss:   455.1108 Validation Accuracy: 0.828125
Epoch  9, Batch 318 -Loss:   356.9030 Validation Accuracy: 0.828125
Epoch  9, Batch 319 -Loss:   322.1104 Validation Accuracy: 0.828125
Epoch  9, Batch 320 -Loss:   277.7638 Validation Accuracy: 0.828125
Epoch  9, Batch 321 -Loss:   361.1666 Validation Accuracy: 0.832031
Epoch  9, Batch 322 -Loss:   309.6254 Validation Accuracy: 0.828125
Epoch  9, Batch 323 -Loss:   179.8538 Validation Accuracy: 0.828125
Epoch  9, Batch 324 -Loss:   233.9950 Validation Accuracy: 0.828125
Epoch  9, Batch 325 -Loss:   275.4861 Validation Accuracy: 0.824219
Epoch  9, Batch 326 -Loss:   182.7706 Validation Accuracy: 0.824219
Epoch  9, Batch 327 -Loss:   397.6231 Validation Accuracy: 0.832031
Epoch  9, Batch 328 -Loss:   334.9595 Validation

Epoch 10, Batch   6 -Loss:   271.1162 Validation Accuracy: 0.828125
Epoch 10, Batch   7 -Loss:   328.0901 Validation Accuracy: 0.828125
Epoch 10, Batch   8 -Loss:   282.1863 Validation Accuracy: 0.828125
Epoch 10, Batch   9 -Loss:   231.7365 Validation Accuracy: 0.828125
Epoch 10, Batch  10 -Loss:   166.4741 Validation Accuracy: 0.828125
Epoch 10, Batch  11 -Loss:   273.5981 Validation Accuracy: 0.828125
Epoch 10, Batch  12 -Loss:   239.3480 Validation Accuracy: 0.832031
Epoch 10, Batch  13 -Loss:   212.0307 Validation Accuracy: 0.832031
Epoch 10, Batch  14 -Loss:   336.1855 Validation Accuracy: 0.828125
Epoch 10, Batch  15 -Loss:   448.9145 Validation Accuracy: 0.828125
Epoch 10, Batch  16 -Loss:   158.8188 Validation Accuracy: 0.828125
Epoch 10, Batch  17 -Loss:   436.7985 Validation Accuracy: 0.832031
Epoch 10, Batch  18 -Loss:   323.1076 Validation Accuracy: 0.832031
Epoch 10, Batch  19 -Loss:   362.0287 Validation Accuracy: 0.832031
Epoch 10, Batch  20 -Loss:   405.1287 Validation

Epoch 10, Batch 127 -Loss:   278.2489 Validation Accuracy: 0.839844
Epoch 10, Batch 128 -Loss:   268.2305 Validation Accuracy: 0.839844
Epoch 10, Batch 129 -Loss:   265.8384 Validation Accuracy: 0.835938
Epoch 10, Batch 130 -Loss:   206.8447 Validation Accuracy: 0.835938
Epoch 10, Batch 131 -Loss:   224.7917 Validation Accuracy: 0.839844
Epoch 10, Batch 132 -Loss:   323.7396 Validation Accuracy: 0.839844
Epoch 10, Batch 133 -Loss:   320.3485 Validation Accuracy: 0.839844
Epoch 10, Batch 134 -Loss:   403.8413 Validation Accuracy: 0.839844
Epoch 10, Batch 135 -Loss:   366.8395 Validation Accuracy: 0.839844
Epoch 10, Batch 136 -Loss:   336.4136 Validation Accuracy: 0.839844
Epoch 10, Batch 137 -Loss:   263.0430 Validation Accuracy: 0.835938
Epoch 10, Batch 138 -Loss:   260.1202 Validation Accuracy: 0.839844
Epoch 10, Batch 139 -Loss:   388.8503 Validation Accuracy: 0.847656
Epoch 10, Batch 140 -Loss:   301.5258 Validation Accuracy: 0.835938
Epoch 10, Batch 141 -Loss:   360.2520 Validation

Epoch 10, Batch 248 -Loss:   254.2195 Validation Accuracy: 0.828125
Epoch 10, Batch 249 -Loss:   152.3969 Validation Accuracy: 0.835938
Epoch 10, Batch 250 -Loss:   263.4873 Validation Accuracy: 0.835938
Epoch 10, Batch 251 -Loss:   368.3826 Validation Accuracy: 0.835938
Epoch 10, Batch 252 -Loss:   193.9507 Validation Accuracy: 0.832031
Epoch 10, Batch 253 -Loss:   530.6735 Validation Accuracy: 0.832031
Epoch 10, Batch 254 -Loss:   351.5011 Validation Accuracy: 0.839844
Epoch 10, Batch 255 -Loss:   233.8175 Validation Accuracy: 0.839844
Epoch 10, Batch 256 -Loss:   269.4659 Validation Accuracy: 0.847656
Epoch 10, Batch 257 -Loss:   271.6493 Validation Accuracy: 0.843750
Epoch 10, Batch 258 -Loss:   214.5795 Validation Accuracy: 0.843750
Epoch 10, Batch 259 -Loss:   269.8224 Validation Accuracy: 0.839844
Epoch 10, Batch 260 -Loss:   205.9576 Validation Accuracy: 0.832031
Epoch 10, Batch 261 -Loss:   273.3139 Validation Accuracy: 0.832031
Epoch 10, Batch 262 -Loss:   123.8621 Validation

Epoch 10, Batch 369 -Loss:   155.0446 Validation Accuracy: 0.835938
Epoch 10, Batch 370 -Loss:   374.7285 Validation Accuracy: 0.835938
Epoch 10, Batch 371 -Loss:   451.8855 Validation Accuracy: 0.832031
Epoch 10, Batch 372 -Loss:   258.9564 Validation Accuracy: 0.828125
Epoch 10, Batch 373 -Loss:   191.1677 Validation Accuracy: 0.832031
Epoch 10, Batch 374 -Loss:   222.6999 Validation Accuracy: 0.828125
Epoch 10, Batch 375 -Loss:   372.0833 Validation Accuracy: 0.832031
Epoch 10, Batch 376 -Loss:   163.3713 Validation Accuracy: 0.832031
Epoch 10, Batch 377 -Loss:   231.0961 Validation Accuracy: 0.828125
Epoch 10, Batch 378 -Loss:   170.8781 Validation Accuracy: 0.828125
Epoch 10, Batch 379 -Loss:   186.5902 Validation Accuracy: 0.828125
Epoch 10, Batch 380 -Loss:   202.2418 Validation Accuracy: 0.828125
Epoch 10, Batch 381 -Loss:   204.3102 Validation Accuracy: 0.828125
Epoch 10, Batch 382 -Loss:   166.4954 Validation Accuracy: 0.828125
Epoch 10, Batch 383 -Loss:   335.4063 Validation